In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import cv2
import pydot

In [4]:
data_dir = "image_final/"
labels = sorted(os.listdir(data_dir))

X = []
y = []
for idx, label in enumerate(labels):
  for file in os.listdir(data_dir + '/'+label):
    filepath = data_dir +'/'+ label + "/" + file
    img = cv2.resize(cv2.imread(filepath),(50,50))
    X.append(img)
    y.append(idx)
X = np.asarray(X)
y = np.asarray(y)
print(X.shape, y.shape)

(4549, 50, 50, 3) (4549,)


In [5]:
#spliting dataset into 80% train, 10% validation and 10% test data
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.8)
X_test, X_eval, Y_test, Y_eval = train_test_split(X_test, Y_test, test_size = 0.5)


In [6]:
# converting Y_tes and Y_train to One hot vectors using to_categorical
# example of one hot => '1' is represented as [0. 1. 0. . . . . 0.]
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
Y_eval = to_categorical(Y_eval)
X_train = X_train / 255.
X_test = X_test/ 255.
X_eval = X_eval/ 255.

In [7]:
# building our model
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu', input_shape=(50,50,3)),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(8, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 16)        2320      
                                                                 
 conv2d_2 (Conv2D)           (None, 44, 44, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 16)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        4640      
                                                                 
 conv2d_4 (Conv2D)           (None, 18, 18, 32)        9248      
                                                        

In [8]:
#compiling the model
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer,
               loss = 'categorical_crossentropy',
               metrics=['accuracy'],)

In [9]:
#start training(fitting) the data
history = model.fit(X_train, Y_train, epochs=100, verbose=1,
                validation_data=(X_eval, Y_eval))

Epoch 1/100
29/29 [==============================] - 4s 103ms/step - loss: 2.0812 - accuracy: 0.1243 - val_loss: 2.0788 - val_accuracy: 0.1324
Epoch 2/100
29/29 [==============================] - 3s 97ms/step - loss: 2.0629 - accuracy: 0.1639 - val_loss: 1.9134 - val_accuracy: 0.2396
Epoch 3/100
29/29 [==============================] - 3s 104ms/step - loss: 1.7731 - accuracy: 0.3344 - val_loss: 1.5582 - val_accuracy: 0.4456
Epoch 4/100
29/29 [==============================] - 3s 98ms/step - loss: 1.6236 - accuracy: 0.4433 - val_loss: 1.5718 - val_accuracy: 0.4242
Epoch 5/100
29/29 [==============================] - 3s 98ms/step - loss: 1.3051 - accuracy: 0.5523 - val_loss: 1.2864 - val_accuracy: 0.5516
Epoch 6/100
29/29 [==============================] - 3s 97ms/step - loss: 1.0488 - accuracy: 0.6392 - val_loss: 1.3728 - val_accuracy: 0.5885
Epoch 7/100
29/29 [==============================] - 3s 103ms/step - loss: 0.9195 - accuracy: 0.6942 - val_loss: 0.8919 - val_accuracy: 0.7313
Epo

In [10]:
#testing
model.evaluate(X_test, Y_test)

57/57 [==============================] - 1s 14ms/step - loss: 1.3093 - accuracy: 0.9000


[1.3093276023864746, 0.8999999761581421]

In [11]:
model.save("model_save/model_final.h5")